## Race and Ethnicity of PGP Particpants

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import sqlite3
import pandas as pd
import numpy as np
import scipy as scipy
import scipy.sparse
import re

from scipy import stats
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.preprocessing import StandardScaler
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D 

In [ ]:
# Set default plot stype for matplotlib
plt.style.use('ggplot')

# Set default figure size for matplotlib
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
X = np.load('/home/swz/PGP-work/Lightning_Work/XfinalPCAAllNewCombine.npy')
#X = np.load('/home/swz/PGP-work/Lightning_Work/XfinalPCAAllNew38.npy')
#plotnamesuffix = '_373819'
plotnamesuffix = '_373819'

In [ ]:
X.shape

In [ ]:
plt.scatter(X[:,0], X[:,1])


In [ ]:
namesfile1 = 'cgf_list37.txt'
namesfile2 = 'cgf_list38.txt'
namesfile3 = 'cgf_listhg19.txt'

names_file1 = open(namesfile1,'r')
names_file2 = open(namesfile2,'r')
names_file3 = open(namesfile3,'r')

names1org = []
names2org = []
names3org = []

for line in names_file1:
    names1org.append(line[:-1])
    np.arange(3)
    
    
for line in names_file2:
    names2org.append(line[:-1])
    np.arange(3)
    
for line in names_file3:
    names3org.append(line[:-1])
    np.arange(3)
    
names = names1org + names2org + names3org
len(names)

In [ ]:
namesarray = np.asarray(names)

namesarray3 = namesarray
names = namesarray3.tolist()
names1 = [i.split('/')[-1] for i in names]
names2 = [i.replace('filtered_','') for i in names1]
names3 = [i.replace('.haplotypeCalls.er.raw','') for i in names2]
names4 = [i.replace('_cg_data_ASM','') for i in names3]
names5 = [i.replace('data_','') for i in names4]
names6 = [i.replace('.cgf','') for i in names5]
names7 = [i.split('_var')[0] for i in names6]
names8 = [i.split('_GS')[0] for i in names7]
names9 = [i.split('_lcl')[0] for i in names8]
names10 = [i.split('_blood')[0] for i in names9]
names11 = [i.split('_buffy')[0] for i in names10]
names12 = [i.split('_noHLA')[0] for i in names11]
names13 = [re.sub('_(S1|sorted).genome','',i) for i in names12]
names14 = [re.sub('_.+-portable', '',i) for i in names13]


In [ ]:
set1 = len(names1org)
set2 = len(names2org)
set3 = len(names3org)

In [ ]:
ancestrySGDP = pd.read_csv('/home/swz/PGP-work/Lightning_Work/SGDP_metadata.txt',encoding='utf8',sep='\t')

In [ ]:
names3org

In [ ]:
ancestry1k =pd.read_excel('/home/swz/PGP-work/Lightning_Work/20130606_sample_info.xlsx')

In [ ]:
ancestry1k.columns

In [ ]:
pcaComponents = pd.DataFrame(np.column_stack([names14, names]),columns=['ID','Filename'])


In [ ]:
pcaComponents['Reference'] =  0
pcaComponents.Reference[0:set1] = '37'
pcaComponents.Reference[set1:set1+set2]  = '38'
pcaComponents.Reference[set1+set2:set1+set2+set3] = 'hg19'

In [ ]:
#pcaComponents =pcaComponents[0:set1+set2]

In [ ]:
ancestryMap = ancestrySGDP[['Illumina_ID','Region']]
ancestryMap['DataSource'] = 'Simons'
np.unique(ancestryMap.Region)
ancestryMap['ID'] = ancestrySGDP.Illumina_ID

ancestryMap2 = ancestry1k[['Sample','Population']]
ancestryMap2['DataSource'] = '1K'
ancestryMap2['ID'] = ancestry1k.Sample
ancestryMap2['Region'] = ancestry1k.Population

ancestries = pd.concat([ancestryMap[['ID','Region','DataSource']], ancestryMap2[['ID','Region','DataSource']]])


In [ ]:
ancestries

In [ ]:
z = pcaComponents.merge(ancestries, how='left', on='ID')

In [ ]:
z.Region.unique()

In [ ]:
z['Color'] = 'black'
# Simons Colors
idxAfrica = z['Region'].str.contains('Africa',na=False)
idxAmerica = z['Region'].str.contains('America',na=False)
idxOceania = z['Region'].str.contains('Oceania',na=False)
idxEastAsia = z['Region'].str.contains('EastAsia',na=False)
idxWestEurasia = z['Region'].str.contains('WestEurasia',na=False)
idxCentralAsiaSiberia = z['Region'].str.contains('CentralAsiaSiberia',na=False)
idxAsiaSouth = z['Region'].str.contains('SouthAsia',na=False)

z.Color[idxAfrica] = 'orange' 
z.Color[idxAmerica] = 'red'
z.Color[idxOceania] = 'yellow'
z.Color[idxEastAsia] = 'blue'
z.Color[idxWestEurasia] = 'pink'
z.Color[idxCentralAsiaSiberia] = 'grey'
z.Color[idxAsiaSouth] = 'purple'


#1K colors
idxAmerica1k = z['Region'].isin(['PUR', 'CLM','MXL','PEL'])
idxEurope1k = z['Region'].isin(['TSI','IBS','CEU','GBR','FIN'])
idxAfrica1k = z['Region'].isin(['LWK','MSL','GWD','YRI','ESN','ACB','ASW'])
idxEastAsia1k = z['Region'].isin(['KHV','CDX','CHS','CHB','JPT'])
idxSouthAsia1k = z['Region'].isin(['STU','ITU','BEB','GIH','PJL'])
z.Color[idxAmerica1k] = 'firebrick'
z.Color[idxEurope1k] = 'green'
z.Color[idxAfrica1k] = 'coral'
z.Color[idxEastAsia1k] = 'royalblue'
z.Color[idxSouthAsia1k] = 'blueviolet'


#PGP colors
idxPGPHarvard = z.ID.str.startswith('hu')
idxPGPCanada = z.ID.str.startswith('PGPC')
z.Color[idxPGPHarvard] = 'darkslategrey'
z.Color[idxPGPCanada] = 'darkolivegreen'

In [ ]:
idx38 = z['Reference'] == '38'
idx37 = z['Reference'] == '37'
idxhg19 = z['Reference'] == 'hg19'

In [ ]:
idxSimons = z.DataSource == 'Simons'
idx1K = z.DataSource == '1K'
idxPGP = ~ (idxSimons | idx1K)


plt.scatter(X[(idxPGP & idx38),0], X[(idxPGP & idx38),1],c=z.Color[(idxPGP & idx38)],marker ="^",s=60, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxPGP & idx37),0], X[(idxPGP & idx37),1],c=z.Color[(idxPGP & idx37)],marker ="o",s=30, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.scatter(X[(idxPGP & idxhg19),0], X[(idxPGP & idxhg19),1],c=z.Color[(idxPGP & idxhg19)],marker ="*",s=80, alpha = 0.8)
xlabel="PCA Component 1"
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.scatter(X[(idxSimons & idx38),0], X[(idxSimons & idx38),1],c=z.Color[(idxSimons & idx38)],marker ='^',s=60, alpha = 0.5)
xlabel="PCA Component 1"
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxSimons & idx37),0], X[(idxSimons & idx37),1],c=z.Color[(idxSimons & idx37)],marker ='o',s=30, alpha = 0.5)
xlabel="PCA Component 1"
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.scatter(X[(idx1K & idx38),0], X[(idx1K & idx38),1],c=z.Color[(idx1K & idx38)],marker ='^',s=60, alpha = 0.5)
xlabel="PCA Component 1"
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idx1K & idx37),0], X[(idx1K & idx37),1],c=z.Color[(idx1K & idx37)],marker ='o',s=30,alpha = 0.5)
xlabel="PCA Component 1"
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)



classes = ['Simons: Africa','1K: Africa', 'Simons: Americas', '1K: Americas','Simons: Oceania', 'Simons: EastAsia', 
           '1K: EastAsia', 'Simons: WestEurasia',
           'Simons: CentralAsiaSiberia', 'Simons: SouthAsia','1K: SouthAsia','1K: Europe','PGP UK', 'Harvard PGP', 'PGP Canada']
class_colours = ['orange','coral','red','firebrick','yellow','blue','royalblue','pink','grey','purple','blueviolet','green' ,'black','darkslategrey', 'darkolivegreen']
recs = []
for i in range(0,len(class_colours)):
    recs.append(mpatches.Rectangle((0,0),1,1,fc=class_colours[i]))
plt.legend(recs,classes,loc='upper right',ncol=2)
plt.title('PCA Decomposition of Tiled Simons Diversity, 1000 Genomes, and PGP Data ')

plotname = 'PCA12' + plotnamesuffix + '.png'
plt.savefig(plotname,format='png', bbox_inches="tight", dpi=300)


In [ ]:
plt.scatter(X[(idxPGP & idx38),0], X[(idxPGP & idx38),1],c=z.Color[(idxPGP & idx38)],marker ="^",s=60, alpha = 0.5)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxPGP & idx37),0], X[(idxPGP & idx37),1],c=z.Color[(idxPGP & idx37)],marker ="o",s=30, alpha = 0.5)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxPGP & idxhg19),0], X[(idxPGP & idxhg19),1],c=z.Color[(idxPGP & idxhg19)],marker ="*",s=80, alpha = 0.5)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.title('PCA Decomposition of PGP Data ')

plotname = 'PCA12PGP' + plotnamesuffix + '.png'
plt.savefig(plotname,format='png', bbox_inches="tight", dpi=300)


In [ ]:
plt.scatter(X[(idxSimons & idx38),0], X[(idxSimons & idx38),1],c=z.Color[(idxSimons & idx38)],marker ="^",s=60, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxSimons & idx37),0], X[(idxSimons & idx37),1],c=z.Color[(idxSimons & idx37)],marker ="o",s=30, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxSimons & idxhg19),0], X[(idxSimons & idxhg19),1],c=z.Color[(idxSimons & idxhg19)],marker ="*",s=30, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.title('PCA Decomposition of Simons Data ')

plotname = 'PCA12Simons' + plotnamesuffix + '.png'
plt.savefig(plotname,format='png', bbox_inches="tight", dpi=300)


In [ ]:
plt.scatter(X[(idx1K & idx38),0], X[(idx1K & idx38),1],c=z.Color[(idx1K & idx38)],marker ="^",s=60, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idx1K & idx37),0], X[(idx1K & idx37),1],c=z.Color[(idx1K & idx37)],marker ="o",s=30, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idx1K & idxhg19),0], X[(idx1K & idxhg19),1],c=z.Color[(idx1K & idxhg19)],marker ="*",s=30, alpha = 0.8)
xlabel="PCA Component 1"#
ylabel="PCA Component 2"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.title('PCA Decomposition of 1000 Genomes Data ')

plotname = 'PCA121K' + plotnamesuffix + '.png'
plt.savefig(plotname,format='png', bbox_inches="tight", dpi=300)

In [ ]:
idxSimons = z.DataSource == 'Simons'
idx1K = z.DataSource == '1K'
idxPGP = ~ (idxSimons | idx1K)

plt.scatter(X[(idxPGP & idx38),1], X[(idxPGP & idx38),2],c=z.Color[(idxPGP & idx38)],marker ="^",s=60, alpha = 0.8)
xlabel="PCA Component 2"#
ylabel="PCA Component 3"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxPGP & idx37),1], X[(idxPGP & idx37),2],c=z.Color[(idxPGP & idx37)],marker ="o",s=30, alpha = 0.8)
xlabel="PCA Component 2"#
ylabel="PCA Component 3"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.scatter(X[(idxSimons & idx38),1], X[(idxSimons & idx38),2],c=z.Color[(idxSimons & idx38)],marker ='^',s=60, alpha = 0.5)
xlabel="PCA Component 2"#
ylabel="PCA Component 3"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idxSimons & idx37),1], X[(idxSimons & idx37),2],c=z.Color[(idxSimons & idx37)],marker ='o',s=30, alpha = 0.5)
xlabel="PCA Component 2"#
ylabel="PCA Component 3"
plt.xlabel(xlabel)
plt.ylabel(ylabel)


plt.scatter(X[(idx1K & idx38),1], X[(idx1K & idx38),2],c=z.Color[(idx1K & idx38)],marker ='^',s=60, alpha = 0.5)
xlabel="PCA Component 2"#
ylabel="PCA Component 3"
plt.xlabel(xlabel)
plt.ylabel(ylabel)

plt.scatter(X[(idx1K & idx37),1], X[(idx1K & idx37),2],c=z.Color[(idx1K & idx37)],marker ='o',s=30,alpha = 0.5)
xlabel="PCA Component 2"#
ylabel="PCA Component 3"
plt.xlabel(xlabel)
plt.ylabel(ylabel)



classes = ['Simons: Africa','1K: Africa', 'Simons: Americas', '1K: Americas','Simons: Oceania', 'Simons: EastAsia', 
           '1K: EastAsia', 'Simons: WestEurasia',
           'Simons: CentralAsiaSiberia', 'Simons: SouthAsia','1K: SouthAsia','1K: Europe','PGP']
class_colours = ['orange','coral','red','firebrick','yellow','blue','royalblue','pink','grey','purple','blueviolet','green' ,'black']
recs = []
for i in range(0,len(class_colours)):
    recs.append(mpatches.Rectangle((0,0),1,1,fc=class_colours[i]))
plt.legend(recs,classes,loc='upper left',ncol=2)
plt.title('PCA Decomposition of Tiled Simons Diversity, 1000 Genomes, and PGP Data ')



plotname = 'PCA23' + plotnamesuffix + '.png'

plt.savefig(plotname,format='png', bbox_inches="tight", dpi=300)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

idxSimons = z.DataSource == 'Simons'
idx1K = z.DataSource == '1K'
idxPGP = ~ (idxSimons | idx1K)


ax.scatter(X[(idxPGP & idx38),0], X[(idxPGP & idx38),1],X[(idxPGP & idx38),2] ,c=z.Color[(idxPGP & idx38)],marker ="^",s=60, alpha = 0.8)
ax.scatter(X[(idxPGP & idx37),0], X[(idxPGP & idx37),1],X[(idxPGP & idx37),2],c=z.Color[(idxPGP & idx37)],marker ="o",s=30, alpha = 0.8)
ax.scatter(X[(idxSimons & idx38),0], X[(idxSimons & idx38),1],X[(idxSimons & idx38),2],c=z.Color[(idxSimons & idx38)],marker ='^',s=60, alpha = 0.5)
ax.scatter(X[(idxSimons & idx37),0], X[(idxSimons & idx37),1],X[(idxSimons & idx37),2],c=z.Color[(idxSimons & idx37)],marker ='o',s=30, alpha = 0.5)
ax.scatter(X[(idx1K & idx38),0],X[(idx1K & idx38),1],X[(idx1K & idx38),2],c=z.Color[(idx1K & idx38)],marker ='^',s=60, alpha = 0.5)
ax.scatter(X[(idx1K & idx37),0],X[(idx1K & idx37),1], X[(idx1K & idx37),2],c=z.Color[(idx1K & idx37)],marker ='o',s=30,alpha = 0.5)




ax.view_init(30, 30)
classes = ['Simons: Africa','1K: Africa', 'Simons: Americas', '1K: Americas','Simons: Oceania', 'Simons: EastAsia', 
           '1K: EastAsia', 'Simons: WestEurasia',
           'Simons: CentralAsiaSiberia', 'Simons: SouthAsia','1K: SouthAsia','1K: Europe','PGP']
class_colours = ['orange','coral','red','firebrick','yellow','blue','royalblue','pink','grey','purple','blueviolet','green' ,'black']
recs = []
for i in range(0,len(class_colours)):
    recs.append(mpatches.Rectangle((0,0),1,1,fc=class_colours[i]))
plt.legend(recs,classes,loc='center left')
plt.title('PCA Decomposition of Tiled Simons Diversity, 1000 Genomes, and PGP Data ')


plotname = 'PCA3D' + plotnamesuffix + '.png'

plt.savefig(plotname,format='png', dpi=300)
